<a href="https://colab.research.google.com/github/johntanas/it1244project/blob/main/code/model_one_hot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [4]:
path_to_evm_label="/content/expo_moving_k2_labels.csv" #https://drive.google.com/file/d/1RUSdg3mKkoY984r4s7rGabjwhJEcIA_2/view?usp=sharing
path_to_app="/content/cleaned_application.csv" #https://drive.google.com/file/d/1Ktdv2NTyK2AM48yBvbpFezd82-srqj7w/view?usp=sharing
cleaned_app_df=pd.read_csv(path_to_app,index_col=0)
ewm_df_status=pd.read_csv(path_to_evm_label,index_col=0)

In [5]:
labels_to_int_evm={"bad":0,"good":1}

In [6]:
ewm_df=ewm_df_status.drop(columns="status").replace({"label":labels_to_int_evm})

In [7]:
dep_var="label"
random_state=42

In [11]:
from sklearn.model_selection import ShuffleSplit
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,export_text,plot_tree
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay

In [12]:
df1=ewm_df.merge(cleaned_app_df,how="inner",on="id").drop("id",axis=1)

In [13]:
df=df1.replace(to_replace = ("Y", "N"),
             value = (1, 0))

In [14]:
gender = pd.get_dummies(df["gender"])
income_type = pd.get_dummies(df["income_type"])
education_level = pd.get_dummies(df["education_level"])
family_status = pd.get_dummies(df["family_status"])
house_type = pd.get_dummies(df["house_type"])
job = pd.get_dummies(df["job"])
one_hot=df.drop(columns = ["gender", "income_type", "education_level",
                   "family_status", "house_type", "job"])
one_hot = pd.concat([one_hot, gender, income_type, education_level, family_status,
                house_type, job], axis = 1)
yvals = pd.DataFrame(one_hot[dep_var])
xvals = one_hot.drop(columns = dep_var)
from sklearn.model_selection import train_test_split
xtrain, xtest, y, valid_y = train_test_split(xvals, yvals, test_size = 0.20,
                                                random_state = random_state)

# Standardizing the training and test data
from sklearn.preprocessing import StandardScaler
std_x = StandardScaler()
std_x.fit(xtrain)
xs = std_x.transform(xtrain)
valid_xs = std_x.transform(xtest)

In [15]:
pd.DataFrame(xs)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.322403,-1.416144,-0.562311,0.253557,0.847678,-0.557377,1.881126,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
1,-0.756199,-1.416144,0.769710,2.175105,0.155277,0.492201,-0.531597,-0.628469,-0.299495,0.891466,...,-0.080549,3.143453,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
2,-0.756199,0.706143,-0.562311,-0.590049,1.100899,0.677858,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
3,1.322403,0.706143,-0.562311,0.394158,0.142709,0.205053,-0.531597,-0.628469,3.338951,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
4,-0.756199,0.706143,-0.562311,1.893903,-1.639846,1.206360,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,2.162272,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6045,-0.756199,0.706143,-0.562311,3.299913,1.210985,0.366368,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6046,-0.756199,-1.416144,-0.562311,0.019222,0.321220,0.262812,-0.531597,-0.628469,-0.299495,-1.261146,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6047,-0.756199,0.706143,-0.562311,1.097163,-1.354042,0.792140,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,2.162272,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6048,-0.756199,-1.416144,-0.562311,-1.058719,1.714634,0.565226,1.881126,-0.628469,-0.299495,-1.261146,...,-0.080549,-0.318122,5.508825,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [17]:
int_to_labels_evm={"bad":1,"good":2}

https://www.kaggle.com/code/umerkk12/credit-card-predictive-analysis#there-are-9516-rows.

In [18]:
classifiers = {
    "LogisticRegression" : LogisticRegression(),
    "KNeighbors" : KNeighborsClassifier(),
    "SVC" : SVC(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "XGBoost" : XGBClassifier()
}

In [19]:
for key, classifier in classifiers.items():
    classifier.fit(xs, y)
    prediction = classifier.predict(valid_xs)
    print("Model "+key+ " using unbalanced data")
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

Model LogisticRegression using unbalanced data
              precision    recall  f1-score   support

         bad       0.67      0.00      0.01       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.70      0.50      0.43      1513
weighted avg       0.71      0.73      0.62      1513

Model KNeighbors using unbalanced data
              precision    recall  f1-score   support

         bad       0.27      0.12      0.16       410
        good       0.73      0.88      0.80      1103

    accuracy                           0.67      1513
   macro avg       0.50      0.50      0.48      1513
weighted avg       0.60      0.67      0.63      1513

Model SVC using unbalanced data
              precision    recall  f1-score   support

         bad       0.50      0.00      0.00       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro a

In [20]:
import keras
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


In [21]:
one_hot_y=pd.get_dummies(y["label"])
one_hot_valid_y=pd.get_dummies(valid_y["label"])

In [22]:
model=keras.Sequential([
keras.Input(shape=(52,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

use BinaryFocalCrossentropy for imbalanced data

In [23]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model.fit(xs, one_hot_y,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
190/190 [==============================] - 4s 12ms/step - loss: 0.1541 - accuracy: 0.7129 - recall: 0.7129 - val_loss: 0.1895 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 2/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1524 - accuracy: 0.7157 - recall: 0.7157 - val_loss: 0.1932 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 3/10
190/190 [==============================] - 2s 10ms/step - loss: 0.1516 - accuracy: 0.7195 - recall: 0.7195 - val_loss: 0.1967 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 4/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1499 - accuracy: 0.7253 - recall: 0.7253 - val_loss: 0.1749 - val_accuracy: 0.4699 - val_recall: 0.4699
Epoch 5/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1508 - accuracy: 0.7235 - recall: 0.7235 - val_loss: 0.1759 - val_accuracy: 0.4501 - val_recall: 0.4501
Epoch 6/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1505 - accuracy: 0

In [24]:
y_prob=model.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.31      0.05      0.09       410
        good       0.73      0.95      0.83      1103

    accuracy                           0.71      1513
   macro avg       0.52      0.50      0.46      1513
weighted avg       0.62      0.71      0.63      1513



In [25]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_balanced, y_balanced = oversample.fit_resample(xs, y)

In [26]:
for key, classifier in classifiers.items():
    classifier.fit(X_balanced, y_balanced)
    prediction = classifier.predict(valid_xs)
    print("Model "+key)
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

Model LogisticRegression
              precision    recall  f1-score   support

         bad       0.27      0.50      0.35       410
        good       0.73      0.49      0.59      1103

    accuracy                           0.49      1513
   macro avg       0.50      0.50      0.47      1513
weighted avg       0.60      0.49      0.52      1513

Model KNeighbors
              precision    recall  f1-score   support

         bad       0.28      0.46      0.35       410
        good       0.73      0.55      0.63      1103

    accuracy                           0.53      1513
   macro avg       0.51      0.51      0.49      1513
weighted avg       0.61      0.53      0.55      1513

Model SVC
              precision    recall  f1-score   support

         bad       0.27      0.44      0.33       410
        good       0.73      0.55      0.63      1103

    accuracy                           0.52      1513
   macro avg       0.50      0.50      0.48      1513
weighted avg       0.6

In [27]:
one_hot_y_balanced=pd.get_dummies(y_balanced["label"])

In [30]:
model2=keras.Sequential([
keras.Input(shape=(52,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

Use BinaryCrossentropy instead

In [31]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model2.fit(X_balanced,one_hot_y_balanced,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
275/275 [==============================] - 4s 11ms/step - loss: 0.6972 - accuracy: 0.5061 - recall_2: 0.5061 - val_loss: 0.6579 - val_accuracy: 0.7290 - val_recall_2: 0.7290
Epoch 2/10
275/275 [==============================] - 3s 10ms/step - loss: 0.6960 - accuracy: 0.5022 - recall_2: 0.5022 - val_loss: 0.6667 - val_accuracy: 0.7211 - val_recall_2: 0.7211
Epoch 3/10
275/275 [==============================] - 3s 10ms/step - loss: 0.6936 - accuracy: 0.5179 - recall_2: 0.5179 - val_loss: 0.7250 - val_accuracy: 0.3047 - val_recall_2: 0.3047
Epoch 4/10
275/275 [==============================] - 3s 10ms/step - loss: 0.6926 - accuracy: 0.5172 - recall_2: 0.5172 - val_loss: 0.6891 - val_accuracy: 0.5155 - val_recall_2: 0.5155
Epoch 5/10
275/275 [==============================] - 3s 11ms/step - loss: 0.6913 - accuracy: 0.5229 - recall_2: 0.5229 - val_loss: 0.7903 - val_accuracy: 0.2723 - val_recall_2: 0.2723
Epoch 6/10
275/275 [==============================] - 3s 10ms/step - loss: 

In [32]:
y_prob=model2.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.28      0.93      0.43       410
        good       0.79      0.10      0.18      1103

    accuracy                           0.33      1513
   macro avg       0.53      0.51      0.30      1513
weighted avg       0.65      0.33      0.25      1513

